Goal: compute the number of slides where a consensus was reached to recut the slide (all three raters labeled the slide Tier 3 or Tier 4)

In [36]:
import pandas as pd
import numpy as np
import enum

In [37]:
path = "/media/jackson/backup/dp_data/tortuosity_study/read_csvs/score_tables/r4_scores.csv"

In [ ]:
from bioptort.analysis.constants import RATER_NAMES


df = pd.read_csv(path)[RATER_NAMES]

In [ ]:
df

In [48]:
arr = df.to_numpy()

In [6]:
def majority_consensus(scores: np.ndarray, target_scores: np.ndarray, consensus_thresh: int) -> bool:
    """
    Check if the number of scores matching the target scores meets or exceeds the consensus threshold.

    Args:
        scores (np.ndarray): Array of scores to check.
        target_scores (np.ndarray): Array of target scores to match against.
        consensus_thresh (int): Minimum number of matches required for consensus.

    Returns:
        bool: True if consensus is reached, False otherwise.
    """
    count = np.isin(scores, target_scores).sum()
    return count >= consensus_thresh


In [41]:
class Tiers(enum.IntEnum):
    TIER_1 = 1
    TIER_2 = 2
    TIER_3 = 3
    TIER_4 = 4

LOW_XY_TORT = [Tiers.TIER_1, Tiers.TIER_3]
HIGH_XY_TORT = [Tiers.TIER_2, Tiers.TIER_4]
LOW_Z_TORT = [Tiers.TIER_1, Tiers.TIER_2]
HIGH_Z_TORT = [Tiers.TIER_3, Tiers.TIER_4]

UNANIMOUS_THRESH = 3
MAJORITY_THRESH = 2

Test majority consensus method

In [45]:
# Example test case
test_scores = [Tiers.TIER_3, Tiers.TIER_3, Tiers.TIER_4]  # Example scores, the last one should be excluded
result = majority_consensus(test_scores, HIGH_Z_TORT, UNANIMOUS_THRESH)
assert result == True, "Test case failed: Expected False but got True"

In [46]:
# Additional test cases for majority_consensus function

# Test case 1: No consensus
test_scores = [Tiers.TIER_1, Tiers.TIER_2, Tiers.TIER_3]
result = majority_consensus(test_scores, HIGH_Z_TORT, UNANIMOUS_THRESH)
assert result == False, "Test case failed: Expected False but got True"

# Test case 2: Partial consensus (below threshold)
test_scores = [Tiers.TIER_3, Tiers.TIER_4, Tiers.TIER_1]
result = majority_consensus(test_scores, HIGH_Z_TORT, UNANIMOUS_THRESH)
assert result == False, "Test case failed: Expected False but got True"

# Test case 3: Exact consensus threshold
test_scores = [Tiers.TIER_3, Tiers.TIER_4, Tiers.TIER_4]
result = majority_consensus(test_scores, HIGH_Z_TORT, UNANIMOUS_THRESH)
assert result == True, "Test case failed: Expected True but got False"

# Test case 4: Above consensus threshold
test_scores = [Tiers.TIER_4, Tiers.TIER_4, Tiers.TIER_4]
result = majority_consensus(test_scores, HIGH_Z_TORT, UNANIMOUS_THRESH)
assert result == True, "Test case failed: Expected True but got False"

print("All test cases passed!")

All test cases passed!


In [47]:
# Test cases for HIGH_XY_TORT

# Test case 1: No consensus
test_scores = [Tiers.TIER_1, Tiers.TIER_3, Tiers.TIER_3]
result = majority_consensus(test_scores, HIGH_XY_TORT, UNANIMOUS_THRESH)
assert result == False, "Test case failed: Expected False but got True"

# Test case 2: Partial consensus (below threshold)
test_scores = [Tiers.TIER_2, Tiers.TIER_4, Tiers.TIER_1]
result = majority_consensus(test_scores, HIGH_XY_TORT, UNANIMOUS_THRESH)
assert result == False, "Test case failed: Expected False but got True"

# Test case 3: Exact consensus threshold
test_scores = [Tiers.TIER_2, Tiers.TIER_4, Tiers.TIER_4]
result = majority_consensus(test_scores, HIGH_XY_TORT, UNANIMOUS_THRESH)
assert result == True, "Test case failed: Expected True but got False"

# Test case 4: Above consensus threshold
test_scores = [Tiers.TIER_4, Tiers.TIER_4, Tiers.TIER_4]
result = majority_consensus(test_scores, HIGH_XY_TORT, UNANIMOUS_THRESH)
assert result == True, "Test case failed: Expected True but got False"

print("All test cases for HIGH_XY_TORT passed!")

All test cases for HIGH_XY_TORT passed!


Calculate the number of images in the final read with a 2/3 consensus

In [53]:
threshhold = MAJORITY_THRESH
target_scores = HIGH_Z_TORT
consensus_slides = [index for index, scores in enumerate(arr) if majority_consensus(
                    scores,
                    target_scores=target_scores, 
                    consensus_thresh=threshhold)
]



Now check 3/3 consensus

In [ ]:
threshhold = UNANIMOUS_THRESH
target_scores = HIGH_Z_TORT
consensus_slides = [index for index, scores in enumerate(arr) if majority_consensus(
                    scores,
                    target_scores=target_scores, 
                    consensus_thresh=threshhold)
]
len(consensus_slides)

102

In [ ]:
threshhold = UNANIMOUS_THRESH
target_scores = LOW_Z_TORT  # Replace with LOW_Z_TORT if needed
consensus_slides = [index for index, scores in enumerate(arr) if majority_consensus(
                    scores,
                    target_scores=target_scores, 
                    consensus_thresh=threshhold)
]
len(consensus_slides)

65

In [ ]:
threshhold = UNANIMOUS_THRESH
target_scores = HIGH_XY_TORT  # Replace with LOW_Z_TORT if needed
consensus_slides = [index for index, scores in enumerate(arr) if majority_consensus(
                    scores,
                    target_scores=target_scores, 
                    consensus_thresh=threshhold)
]
len(consensus_slides)

27

In [69]:
threshhold = UNANIMOUS_THRESH
target_scores = LOW_XY_TORT  # Replace with LOW_Z_TORT if needed
consensus_slides = [index for index, scores in enumerate(arr) if majority_consensus(
                    scores,
                    target_scores=target_scores, 
                    consensus_thresh=threshhold)
]
len(consensus_slides)

103

In [72]:
consensus_results = []
thresholds = {
    "UNANIMOUS_THRESH": UNANIMOUS_THRESH,
    "MAJORITY_THRESH": MAJORITY_THRESH
}
target_scores_list = {
    "HIGH_Z_TORT": HIGH_Z_TORT,
    "LOW_Z_TORT": LOW_Z_TORT,
    "HIGH_XY_TORT": HIGH_XY_TORT,
    "LOW_XY_TORT": LOW_XY_TORT,
}

for threshold_key, threshold_value in thresholds.items():
    for target_scores_key, target_scores_values in target_scores_list.items():
        consensus_slides = [index for index, scores in enumerate(arr) if majority_consensus(
                            scores,
                            target_scores=target_scores_values, 
                            consensus_thresh=threshold_value)
        ]
        consensus_results.append({
            "threshold": threshold_key,
            "target_scores": target_scores_key,
            "count": len(consensus_slides)
        })

for result in consensus_results:
    print(f"Threshold: {result['threshold']}, Target Scores: {result['target_scores']}, Count: {result['count']}")

Threshold: UNANIMOUS_THRESH, Target Scores: HIGH_Z_TORT, Count: 90
Threshold: UNANIMOUS_THRESH, Target Scores: LOW_Z_TORT, Count: 60
Threshold: UNANIMOUS_THRESH, Target Scores: HIGH_XY_TORT, Count: 15
Threshold: UNANIMOUS_THRESH, Target Scores: LOW_XY_TORT, Count: 103
Threshold: MAJORITY_THRESH, Target Scores: HIGH_Z_TORT, Count: 102
Threshold: MAJORITY_THRESH, Target Scores: LOW_Z_TORT, Count: 65
Threshold: MAJORITY_THRESH, Target Scores: HIGH_XY_TORT, Count: 27
Threshold: MAJORITY_THRESH, Target Scores: LOW_XY_TORT, Count: 140
